### Data Processing

In [1]:
import spacy
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
df = pd.read_pickle('C:/Users/Ana/OneDrive - Hochschule Düsseldorf/MA/data_merged.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 971264 entries, 0 to 971279
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   text            971264 non-null  object        
 1   date            971264 non-null  datetime64[ns]
 2   speaker_name    971264 non-null  object        
 3   speaker_gender  971264 non-null  object        
 4   speaker_role    884850 non-null  object        
 5   speaker_party   969563 non-null  object        
 6   comments        971264 non-null  object        
 7   id              971264 non-null  object        
 8   speaker_id      971264 non-null  int64         
 9   speech_length   971264 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(7)
memory usage: 81.5+ MB


In [ ]:
#!python -m spacy download de_core_news_sm # downloaded successfully

In [3]:
nlp = spacy.load('de_core_news_sm') 
doc = nlp(df.text[0])
doc

  Sehr geehrter Herr Alterspräsident! So muss ich es jetzt sagen, genau; ich muss mich daran gewöhnen. Nicht lange.  Nicht lange; da haben Sie schon recht. – Liebe Kolleginnen und Kollegen! Meine sehr geehrten Damen und Herren! Die Übernahme der Geschäftsordnung ist eine der ersten wichtigen Entscheidungen, die wir heute zu Beginn der Wahlperiode treffen müssen. Die Geschäftsordnung ist Grundlage für unsere gemeinsame Arbeit. Sie hat sich über viele Wahlperioden bewährt und wird uns auch durch die neue Wahlperiode tragen. Es ist gute Tradition, dass wir die Geschäftsordnung am Anfang einer Wahlperiode mit breiter parlamentarischer Mehrheit übernehmen, und das sollten wir auch heute tun. Sie ist nicht in Stein gemeißelt – das wissen diejenigen, die bereits in der letzten Wahlperiode hier im Bundestag tätig waren –, sondern die Geschäftsordnung ist immer ein, ich sage mal, lebendiges Dokument, das sich den Gegebenheiten des Parlamentarismus, aber auch den Gegebenheiten der Arbeit hier im

In [4]:
def keep_token(token):
    exclude_pos = ['ADV', 'PRON', 'CCONJ', 'PUNCT', 'PART', 'DET', 'ADP','SPACE', 'NUM', 'SYM']
    exclude_tokens = ['kollege', 'kollegin', 'abgeordneter', 'redner', 'rednerin', 'staatssekretär', 'staatssekretärin', 'minister', 'ministerin', 'bundesminister']

    if token.is_alpha and token.ent_type_ != 'PER' and not (token.is_stop or token.pos_ in exclude_pos or token.lemma_.lower() in exclude_tokens):
       return True
    return False

tokens = [token.lemma_.lower() for token in doc if keep_token(token)] # Nur zur Ausgabe
' '.join(tokens)

'geehrt herr alterspräsident sagen gewöhnen lieb geehrt dame herr übernahme geschäftsordnung wichtig entscheidung beginn wahlperiode treffen geschäftsordnung grundlage gemeinsam arbeit wahlperiode bewähren wahlperiode tragen tradition geschäftsordnung anfang wahlperiode breit parlamentarisch mehrheit übernehmen stein meißeln wissen letzter wahlperiode bundestag geschäftsordnung sagen lebendig dokument gegebenheit parlamentarismus gegebenheit arbeit deutsch bundestag anpassen geschäftsordnung entwicklung nachvollziehen letzter wahlperiode beweisen grundlegend änderung beschließen vorangestellen überfällig reform verhaltensregeln begrüßen initiative scheidend präsident geschäftsordnung wahlperiode generalüberholung unterziehen impuls aufnehmen laden fraktion weg vorschlag opposition nehmen möchten aufgreifen möchten diskutieren geschäftsordnung überarbeitet nächster wirken plädieren letzter legislaturperiode eingeführt regelung digital ausschuß beibehalten antrag entnehmen plädieren eing

In [5]:
speeches_processed = []

for speech in tqdm(nlp.pipe(df.text, n_process=6, batch_size=64)):
    tokens = [token.lemma_.lower() for token in speech if keep_token(token)]
    speeches_processed.append(tokens)

0it [00:00, ?it/s]

In [6]:
len(speeches_processed)

971264

#### Save processed speeches

In [1]:
import pickle

# Save speeches_processed to a file
#with open('C:/Users/Ana/OneDrive - Hochschule Düsseldorf/MA/speeches_processed.pkl', 'wb') as f:
 #   pickle.dump(speeches_processed, f)

# Load speeches_processed from the file
with open('C:/Users/Ana/OneDrive - Hochschule Düsseldorf/MA/speeches_processed.pkl', 'rb') as f:
   loaded_speeches = pickle.load(f)

In [2]:
print(loaded_speeches[0])

['geehrt', 'herr', 'alterspräsident', 'sagen', 'gewöhnen', 'lieb', 'geehrt', 'dame', 'herr', 'übernahme', 'geschäftsordnung', 'wichtig', 'entscheidung', 'beginn', 'wahlperiode', 'treffen', 'geschäftsordnung', 'grundlage', 'gemeinsam', 'arbeit', 'wahlperiode', 'bewähren', 'wahlperiode', 'tragen', 'tradition', 'geschäftsordnung', 'anfang', 'wahlperiode', 'breit', 'parlamentarisch', 'mehrheit', 'übernehmen', 'stein', 'meißeln', 'wissen', 'letzter', 'wahlperiode', 'bundestag', 'geschäftsordnung', 'sagen', 'lebendig', 'dokument', 'gegebenheit', 'parlamentarismus', 'gegebenheit', 'arbeit', 'deutsch', 'bundestag', 'anpassen', 'geschäftsordnung', 'entwicklung', 'nachvollziehen', 'letzter', 'wahlperiode', 'beweisen', 'grundlegend', 'änderung', 'beschließen', 'vorangestellen', 'überfällig', 'reform', 'verhaltensregeln', 'begrüßen', 'initiative', 'scheidend', 'präsident', 'geschäftsordnung', 'wahlperiode', 'generalüberholung', 'unterziehen', 'impuls', 'aufnehmen', 'laden', 'fraktion', 'weg', 'vor

In [3]:
len(loaded_speeches)

971264

### Topic Modelling

In [ ]:
#!pip install gensim

In [4]:
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel

In [5]:
dictionary = Dictionary(loaded_speeches)

In [6]:
dictionary.num_docs

971264

In [7]:
dictionary.filter_extremes(no_below=20, no_above=0.3) #, keep_n=1000)

In [8]:
len(dictionary)

85488

In [9]:
dictionary.token2id

{'absenken': 0,
 'alterspräsident': 1,
 'anfang': 2,
 'anpassen': 3,
 'antrag': 4,
 'arbeit': 5,
 'aufgreifen': 6,
 'aufnehmen': 7,
 'ausschuß': 8,
 'beginn': 9,
 'begrüßen': 10,
 'beibehalten': 11,
 'beschließen': 12,
 'beschlussfähigkeit': 13,
 'beweisen': 14,
 'bewähren': 15,
 'bitte': 16,
 'bitten': 17,
 'breit': 18,
 'bundestag': 19,
 'coronabedingt': 20,
 'dame': 21,
 'deutsch': 22,
 'digital': 23,
 'diskutieren': 24,
 'dokument': 25,
 'eingeführt': 26,
 'entnehmen': 27,
 'entscheidung': 28,
 'entwicklung': 29,
 'erforderlich': 30,
 'fraktion': 31,
 'geehrt': 32,
 'gegebenheit': 33,
 'gemeinsam': 34,
 'generalüberholung': 35,
 'geschäftsordnung': 36,
 'gewöhnen': 37,
 'grundlage': 38,
 'grundlegend': 39,
 'herzlich': 40,
 'impuls': 41,
 'initiative': 42,
 'laden': 43,
 'lebendig': 44,
 'legislaturperiode': 45,
 'letzter': 46,
 'lieb': 47,
 'mehrheit': 48,
 'meißeln': 49,
 'möchten': 50,
 'nachvollziehen': 51,
 'nehmen': 52,
 'notwendig': 53,
 'nächster': 54,
 'opposition': 55,
 '

In [10]:
corpus = [dictionary.doc2bow(doc) for doc in loaded_speeches]

In [13]:
lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=50,
                         num_topics=30, workers=8, passes=15)

In [12]:
lda_model.print_topics()

[(0,
  '0.341*"frau" + 0.060*"zwischenfrage" + 0.046*"gestatten" + 0.036*"nachfrage" + 0.031*"redezeit" + 0.021*"präsidentin" + 0.016*"nächste" + 0.015*"schluss" + 0.014*"stellen" + 0.014*"minute"'),
 (1,
  '0.046*"milliarde" + 0.038*"dm" + 0.035*"million" + 0.029*"haushalt" + 0.012*"bund" + 0.011*"land" + 0.011*"geld" + 0.009*"zahl" + 0.008*"höhe" + 0.008*"ausgabe"'),
 (2,
  '0.023*"sagen" + 0.012*"dame" + 0.011*"wissen" + 0.009*"sprechen" + 0.008*"spd" + 0.007*"debatte" + 0.007*"lassen" + 0.007*"reden" + 0.006*"regierung" + 0.006*"letzter"'),
 (3,
  '0.011*"energie" + 0.008*"erneuerbar" + 0.007*"klimaschutz" + 0.007*"forschung" + 0.007*"bundesregierung" + 0.006*"deutschland" + 0.006*"umwelt" + 0.005*"ökologisch" + 0.005*"technologie" + 0.004*"ziel"'),
 (4,
  '0.011*"rente" + 0.011*"arbeitnehmer" + 0.011*"arbeit" + 0.009*"sozial" + 0.009*"gesetz" + 0.007*"gesetzentwurf" + 0.007*"gesetzlich" + 0.007*"leistung" + 0.006*"einkommen" + 0.006*"rentenversicherung"'),
 (5,
  '0.030*"bundesweh